In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import MultiLabelBinarizer
#from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers

In [2]:
df = pd.read_csv('data/single_step_df_ints_2022-03-30.csv')

In [3]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,plate.already_seen,blueberries.already_seen,drink.already_seen,plate_small.already_seen,bottle.already_seen,...,food.food_k,food.strong_k,food.mid_k,coordinates_drink.x,coordinates_drink.y,coordinates_drink.z,drink.containment,drink.food_k,drink.strong_k,drink.mid_k
0,<start>,plate,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,plate,plate_small,0.513000,-0.531000,0.740000,1.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,plate_small,cup,0.513000,-0.531000,0.740000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cup,glass,0.513000,-0.531000,0.740000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,glass,knife,0.513000,-0.531000,0.740000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,drink,drink,0.000000,4.000000,1.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0
1457,drink,spoon,0.000000,4.000000,1.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0
1458,spoon,cereal,0.000000,2.000000,2.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0
1459,cereal,glass,0.000000,1.000000,3.000000,0.0,0.0,1.0,0.0,0.0,...,NaN,NaN,NaN,0.0,4.0,1.0,1.0,0.0,0.0,0.0


In [4]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [5]:
df['target'].unique()
df['target'].value_counts()

plate            186
glass            164
knife            162
fork             157
cup              148
spoon            138
bowl              82
wineglass         74
plate_small       73
bottle            72
spoon_small       47
drink             26
fork_small        22
tray              22
silverware        21
cereal            15
placemat          10
napkin             9
milk               4
salad              4
table_cloth        3
water              3
plate2             2
food               2
yoghurt            2
cheese             1
bread              1
beer               1
oil                1
blueberries        1
tupperware         1
apricots           1
bananas            1
tomato_plate       1
can                1
cutting_board      1
container          1
grapes             1
Name: target, dtype: int64

In [6]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [7]:
# shuffle

df_new = df.sample(frac=1)

df_new

,input,target,start_coords.x,start_coords.y,start_coords.z,plate.already_seen,blueberries.already_seen,drink.already_seen,plate_small.already_seen,bottle.already_seen,...,food.food_k,food.strong_k,food.mid_k,coordinates_drink.x,coordinates_drink.y,coordinates_drink.z,drink.containment,drink.food_k,drink.strong_k,drink.mid_k
497,cup,glass,0.513,-0.531,0.74,1.0,0.0,0.0,1.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1378,spoon,cup,5.500,4.000,2.00,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
929,fork,cup,5.500,4.000,2.00,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
702,silverware,cup,4.000,3.000,2.00,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
798,<start>,placemat,3.000,4.000,1.00,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1333,<start>,plate,2.500,6.500,2.00,0.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1290,spoon,cup,5.500,4.000,2.00,1.0,0.0,0.0,0.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0
1452,spoon,glass,0.000,2.000,2.00,0.0,0.0,1.0,0.0,0.0,...,-99.0,-99.0,-99.0,0.0,4.0,1.0,1.0,0.0,0.0,0.0
465,spoon,knife,0.513,-0.531,0.74,1.0,0.0,0.0,1.0,0.0,...,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0,-99.0


In [8]:
# generate stratified split for train - test/val

#split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

#for train_index, test_index in split.split(df_new, df_new['target']):
#    strat_train = df_new.loc[train_index]
#    strat_test_val = df_new.loc[test_index]

In [9]:
train = df[:int(round(len(df)*0.7,0))]
test_val = df[int(round(len(df)*0.7,0)):]

In [10]:
# split test_val into test and val

split_test_val = len(test_val)//2

test = test_val[:split_test_val]
val = test_val[split_test_val:]

In [11]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [12]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [13]:
batch_size = 256

In [15]:
train_ds = create_dataset(train, labels, batch_size=batch_size)
val_ds = create_dataset(val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_23489/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


In [39]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of targets:', label_batch)

Every feature: ['input', 'start_coords.x', 'start_coords.y', 'start_coords.z', 'plate.already_seen', 'blueberries.already_seen', 'drink.already_seen', 'plate_small.already_seen', 'bottle.already_seen', 'wineglass.already_seen', 'napkin.already_seen', 'tray.already_seen', 'silverware.already_seen', 'water.already_seen', 'apricots.already_seen', 'tupperware.already_seen', 'beer.already_seen', 'cutting_board.already_seen', 'grapes.already_seen', 'table_cloth.already_seen', 'cheese.already_seen', 'cereal.already_seen', 'spoon_small.already_seen', 'tomato_plate.already_seen', 'food.already_seen', 'milk.already_seen', 'oil.already_seen', 'bananas.already_seen', 'fork_small.already_seen', 'salad.already_seen', 'bread.already_seen', 'can.already_seen', 'plate2.already_seen', 'container.already_seen', 'placemat.already_seen', 'spoon.already_seen', 'yoghurt.already_seen', 'bowl.already_seen', 'knife.already_seen', 'cup.already_seen', 'fork.already_seen', 'glass.already_seen', 'coordinates_plate.

In [16]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [17]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [18]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [19]:
all_inputs, encoded_features = create_input_data(df)

In [20]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10)

In [21]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(38)(x)

model = tf.keras.Model(all_inputs, output)

In [22]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [23]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
4/4 [==============================] - 12s 2s/step - loss: 0.0000e+00 - accuracy: 0.0381 - val_loss: 0.0000e+00 - val_accuracy: 0.3105
Epoch 2/100
4/4 [==============================] - 0s 107ms/step - loss: 0.0000e+00 - accuracy: 0.0704 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/100
4/4 [==============================] - 0s 107ms/step - loss: 0.0000e+00 - accuracy: 0.0244 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/100
4/4 [==============================] - 0s 106ms/step - loss: 0.0000e+00 - accuracy: 0.0342 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/100
4/4 [==============================] - 0s 108ms/step - loss: 0.0000e+00 - accuracy: 0.0293 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 6/100
4/4 [==============================] - 0s 108ms/step - loss: 0.0000e+00 - accuracy: 0.0313 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 7/100
4/4 [==============================] - 0s 107ms/step - loss: 0.0000e+00 - ac

In [24]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 173ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
Accuracy 0.0


In [28]:
model.save('models/next_obj_classifier_2022-03-23')

2022-03-23 12:41:32.131340: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_2022-03-23/assets


In [31]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_2022-03-23')

In [ ]:
# test prediction for one sample

In [32]:
sample = df.loc[0].drop('target').to_dict()

In [33]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [34]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [35]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [36]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in range(0, len(dataframe)): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors

In [37]:
errors = get_prequential_error(df, model, labels, nr_of_sequences)

In [38]:
summed_error = [sum(error) for error in errors[:-1]]

In [39]:
np.median(summed_error)

2.0

In [41]:
#with open('results/nn_spatialinfo_prequential_summed_2022-03-23.txt', 'w') as file:
#    file.write(str(summed_error))